In [ ]:
DEBUG = False

In [ ]:
import os
import sys
sys.path = [
    '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
] + sys.path

In [ ]:
import skimage.io
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from efficientnet_pytorch import model as enet

import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm


In [ ]:
data_dir = '../input/prostate-cancer-grade-assessment'
df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
df_test = pd.read_csv(os.path.join(data_dir, 'test.csv'))
df_sub = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))

model_dir = '../input/panda-public-models'
image_folder = os.path.join(data_dir, 'test_images')
is_test = os.path.exists(image_folder)  # IF test_images is not exists, we will use some train images.
image_folder = image_folder if is_test else os.path.join(data_dir, 'train_images')

df = df_test if is_test else df_train.loc[:10]

tile_size = 256
image_size = 256
n_tiles = 36
batch_size = 8
num_workers = 4

device = torch.device('cuda')
print(image_folder)


# Model

In [ ]:
class enetv2(nn.Module):
    def __init__(self, backbone, out_dim):
        super(enetv2, self).__init__()
        self.enet = enet.EfficientNet.from_name(backbone)
        self.myfc = nn.Linear(self.enet._fc.in_features, out_dim)
        self.enet._fc = nn.Identity()

    def extract(self, x):
        return self.enet(x)

    def forward(self, x):
        x = self.extract(x)
        x = self.myfc(x)
        return x
    
    
def load_models(model_files):
    models = []
    for model_f in model_files:
        model_f = os.path.join(model_dir, model_f)
        backbone = 'efficientnet-b0'
        model = enetv2(backbone, out_dim=5)
        model.load_state_dict(torch.load(model_f, map_location=lambda storage, loc: storage), strict=True)
        model.eval()
        model.to(device)
        models.append(model)
        print(f'{model_f} loaded!')
    return models


model_files = [
    'cls_effnet_b0_Rand36r36tiles256_big_bce_lr0.3_augx2_30epo_model_fold0.pth'
]

models = load_models(model_files)



model_dir2 = '../input/pandaenetb042x256x256x3'
def load_models2(model_files):
    models = []
    for model_f in model_files:
        model_f = os.path.join(model_dir2, model_f)
        backbone = 'efficientnet-b1'
        model = enetv2(backbone, out_dim=5)
        model.load_state_dict(torch.load(model_f, map_location=lambda storage, loc: storage), strict=True)
        model.eval()
        model.to(device)
        models.append(model)
        print(f'{model_f} loaded!')
    return models


model_files2 = [
    'enet_b1_8ep_fold0.pth'
]

models2 = load_models2(model_files2)

# Dataset

In [ ]:
def get_tiles(img, mode=0):
        result = []
        h, w, c = img.shape
        pad_h = (tile_size - h % tile_size) % tile_size + ((tile_size * mode) // 2)
        pad_w = (tile_size - w % tile_size) % tile_size + ((tile_size * mode) // 2)

        img2 = np.pad(img,[[pad_h // 2, pad_h - pad_h // 2], [pad_w // 2,pad_w - pad_w//2], [0,0]], constant_values=255)
        img3 = img2.reshape(
            img2.shape[0] // tile_size,
            tile_size,
            img2.shape[1] // tile_size,
            tile_size,
            3
        )

        img3 = img3.transpose(0,2,1,3,4).reshape(-1, tile_size, tile_size,3)
        n_tiles_with_info = (img3.reshape(img3.shape[0],-1).sum(1) < tile_size ** 2 * 3 * 255).sum()
        if len(img) < n_tiles:
            img3 = np.pad(img3,[[0,n_tiles-len(img3)],[0,0],[0,0],[0,0]], constant_values=255)
        idxs = np.argsort(img3.reshape(img3.shape[0],-1).sum(-1))[:n_tiles]
        img3 = img3[idxs]
        for i in range(len(img3)):
            result.append({'img':img3[i], 'idx':i})
        return result, n_tiles_with_info >= n_tiles


class PANDADataset(Dataset):
    def __init__(self,
                 df,
                 image_size,
                 n_tiles=n_tiles,
                 tile_mode=0,
                 rand=False,
                 sub_imgs=False,
                 transform=None
                ):

        self.df = df.reset_index(drop=True)
        self.image_size = image_size
        self.n_tiles = n_tiles
        self.tile_mode = tile_mode
        self.rand = rand
        self.sub_imgs = sub_imgs
        self.transform = transform

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_id = row.image_id
        
        tiff_file = os.path.join(image_folder, f'{img_id}.tiff')
        image = skimage.io.MultiImage(tiff_file)[1]
        tiles, OK = get_tiles(image, self.tile_mode)

        if self.rand:
            idxes = np.random.choice(list(range(self.n_tiles)), self.n_tiles, replace=False)
        else:
            idxes = list(range(self.n_tiles))
        idxes = np.asarray(idxes) + self.n_tiles if self.sub_imgs else idxes

        n_row_tiles = int(np.sqrt(self.n_tiles))
        images = np.zeros((image_size * n_row_tiles, image_size * n_row_tiles, 3))
        for h in range(n_row_tiles):
            for w in range(n_row_tiles):
                i = h * n_row_tiles + w
    
                if len(tiles) > idxes[i]:
                    this_img = tiles[idxes[i]]['img']
                else:
                    this_img = np.ones((self.image_size, self.image_size, 3)).astype(np.uint8) * 255
                this_img = 255 - this_img
                h1 = h * image_size
                w1 = w * image_size
                images[h1:h1+image_size, w1:w1+image_size] = this_img

        if self.transform is not None:
            images = self.transform(image=images)['image']
            
        images = images.astype(np.float32)
        images /= 255
        images = images.transpose(2, 0, 1)

        return torch.tensor(images)


In [ ]:
if not is_test:
    dataset_show = PANDADataset(df, image_size, n_tiles, 0)
    from pylab import rcParams
    rcParams['figure.figsize'] = 20,10
    for i in range(2):
        f, axarr = plt.subplots(1,5)
        for p in range(5):
            idx = np.random.randint(0, len(dataset_show))
            img = dataset_show[idx]
            axarr[p].imshow(1. - img.transpose(0, 1).transpose(1,2).squeeze())
            axarr[p].set_title(str(idx))

# Prediction

In [ ]:
import albumentations

transforms_train = albumentations.Compose([
    albumentations.Transpose(p=0.5),
    albumentations.VerticalFlip(p=0.5),
    albumentations.HorizontalFlip(p=0.5),
])


transforms_val = albumentations.Compose([])

transforms_val1 = albumentations.Compose([
    albumentations.Transpose(p=1)
])

transforms_val2 = albumentations.Compose([
    albumentations.VerticalFlip(p=1)
])

transforms_val3= albumentations.Compose([
    albumentations.HorizontalFlip(p=1),
])

transforms_val4= albumentations.Compose([
    albumentations.Transpose(p=1),
    albumentations.VerticalFlip(p=1),
    albumentations.HorizontalFlip(p=1),
])

In [ ]:

dataset = PANDADataset(df, image_size, n_tiles, 0, False, False, transforms_val )  # mode == 0
loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

dataset2 = PANDADataset(df, image_size, n_tiles, 2, False, False, transforms_val )  # mode == 2
loader2 = DataLoader(dataset2, batch_size=batch_size, num_workers=num_workers, shuffle=False)


LOGITS = []
LOGITS2 = []
LOGITS3 = []
LOGITS4 = []
with torch.no_grad():
    for data in tqdm(loader):
        data = data.to(device)
        logits = models[0](data)
        LOGITS.append(logits)
        logits = models2[0](data)
        LOGITS3.append(logits)
        
    for data in tqdm(loader2):
        data = data.to(device)
        logits = models[0](data)
        LOGITS2.append(logits)
        logits = models2[0](data)
        LOGITS4.append(logits)
        
LOGITS = (torch.cat(LOGITS).sigmoid().cpu()+torch.cat(LOGITS2).sigmoid().cpu()+torch.cat(LOGITS3).sigmoid().cpu()+torch.cat(LOGITS4).sigmoid().cpu()) / 4
PREDS = LOGITS.sum(1).numpy()



In [ ]:

dataset = PANDADataset(df, image_size, n_tiles, 0, False, False, transforms_val1 )  # mode == 0
loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

dataset2 = PANDADataset(df, image_size, n_tiles, 2, False, False, transforms_val1 )  # mode == 2
loader2 = DataLoader(dataset2, batch_size=batch_size, num_workers=num_workers, shuffle=False)


LOGITS = []
LOGITS2 = []
LOGITS3 = []
LOGITS4 = []
with torch.no_grad():
    for data in tqdm(loader):
        data = data.to(device)
        logits = models[0](data)
        LOGITS.append(logits)
        logits = models2[0](data)
        LOGITS3.append(logits)
        
    for data in tqdm(loader2):
        data = data.to(device)
        logits = models[0](data)
        LOGITS2.append(logits)
        logits = models2[0](data)
        LOGITS4.append(logits)
        
LOGITS = (torch.cat(LOGITS).sigmoid().cpu()+torch.cat(LOGITS2).sigmoid().cpu()+torch.cat(LOGITS3).sigmoid().cpu()+torch.cat(LOGITS4).sigmoid().cpu()) / 4
PREDS1 = LOGITS.sum(1).numpy()


In [ ]:
dataset = PANDADataset(df, image_size, n_tiles, 0, False, False, transforms_val2 )  # mode == 0
loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

dataset2 = PANDADataset(df, image_size, n_tiles, 2, False, False, transforms_val2 )  # mode == 2
loader2 = DataLoader(dataset2, batch_size=batch_size, num_workers=num_workers, shuffle=False)


LOGITS = []
LOGITS2 = []
LOGITS3 = []
LOGITS4 = []
with torch.no_grad():
    for data in tqdm(loader):
        data = data.to(device)
        logits = models[0](data)
        LOGITS.append(logits)
        logits = models2[0](data)
        LOGITS3.append(logits)
        
    for data in tqdm(loader2):
        data = data.to(device)
        logits = models[0](data)
        LOGITS2.append(logits)
        logits = models2[0](data)
        LOGITS4.append(logits)
        
LOGITS = (torch.cat(LOGITS).sigmoid().cpu()+torch.cat(LOGITS2).sigmoid().cpu()+torch.cat(LOGITS3).sigmoid().cpu()+torch.cat(LOGITS4).sigmoid().cpu()) / 4
PREDS2 = LOGITS.sum(1).numpy()


In [ ]:
dataset = PANDADataset(df, image_size, n_tiles, 0, False, False, transforms_val3 )  # mode == 0
loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

dataset2 = PANDADataset(df, image_size, n_tiles, 2, False, False, transforms_val3 )  # mode == 2
loader2 = DataLoader(dataset2, batch_size=batch_size, num_workers=num_workers, shuffle=False)


LOGITS = []
LOGITS2 = []
LOGITS3 = []
LOGITS4 = []
with torch.no_grad():
    for data in tqdm(loader):
        data = data.to(device)
        logits = models[0](data)
        LOGITS.append(logits)
        logits = models2[0](data)
        LOGITS3.append(logits)
        
    for data in tqdm(loader2):
        data = data.to(device)
        logits = models[0](data)
        LOGITS2.append(logits)
        logits = models2[0](data)
        LOGITS4.append(logits)
        
LOGITS = (torch.cat(LOGITS).sigmoid().cpu()+torch.cat(LOGITS2).sigmoid().cpu()+torch.cat(LOGITS3).sigmoid().cpu()+torch.cat(LOGITS4).sigmoid().cpu()) / 4
PREDS3 = LOGITS.sum(1).numpy()

In [ ]:
dataset = PANDADataset(df, image_size, n_tiles, 0, False, False, transforms_val4 )  # mode == 0
loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)

dataset2 = PANDADataset(df, image_size, n_tiles, 2, False, False, transforms_val4 )  # mode == 2
loader2 = DataLoader(dataset2, batch_size=batch_size, num_workers=num_workers, shuffle=False)


LOGITS = []
LOGITS2 = []
LOGITS3 = []
LOGITS4 = []
with torch.no_grad():
    for data in tqdm(loader):
        data = data.to(device)
        logits = models[0](data)
        LOGITS.append(logits)
        logits = models2[0](data)
        LOGITS3.append(logits)
        
    for data in tqdm(loader2):
        data = data.to(device)
        logits = models[0](data)
        LOGITS2.append(logits)
        logits = models2[0](data)
        LOGITS4.append(logits)
        
LOGITS = (torch.cat(LOGITS).sigmoid().cpu()+torch.cat(LOGITS2).sigmoid().cpu()+torch.cat(LOGITS3).sigmoid().cpu()+torch.cat(LOGITS4).sigmoid().cpu()) / 4
PREDS4 = LOGITS.sum(1).numpy()


In [ ]:
!pip install ../input/kaggle-efficientnet-repo/efficientnet-1.0.0-py3-none-any.whl

import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import argparse
import os
import skimage.io
from scipy.ndimage import measurements
import os
import numpy as np
import pandas as pd
import argparse
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy
from kaggle_datasets import KaggleDatasets
from tensorflow.keras import layers as L
import efficientnet.tfkeras as efn
from tensorflow.keras.utils import to_categorical
import gc
import albumentations
gc.enable()



sz = 256
N = 48
def tile(img):
    result = []
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    img = img[idxs]
    return img

def tile2(img):
    result = []
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz + ((sz * 2) // 2), (sz - shape[1]%sz)%sz + ((sz * 2) // 2)
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    img = img[idxs]
    return img




class ConvNet(tf.keras.Model):

    def __init__(self, engine, input_shape, weights):
        super(ConvNet, self).__init__()
        
        self.engine = engine(
            include_top=False, input_shape=input_shape, weights=weights)
        
        
        self.avg_pool2d = tf.keras.layers.GlobalAveragePooling2D()
        self.dropout = tf.keras.layers.Dropout(0.5)
        self.dense_1 = tf.keras.layers.Dense(1024)
        self.dense_2 = tf.keras.layers.Dense(5,activation='sigmoid')

    @tf.function
    def call(self, inputs, **kwargs):
        x = tf.reshape(inputs, (-1, IMG_SIZE, IMG_SIZE, 3))
        x = self.engine(x)
        shape = x.shape
        x = tf.reshape(x, (-1, N_TILES, shape[1], shape[2], shape[3])) 
        x = tf.transpose(x, perm=[0, 2, 1, 3, 4])
        x = tf.reshape(x, (-1, shape[1], N_TILES*shape[2], shape[3])) 
        x = self.avg_pool2d(x)
        x = self.dropout(x, training=False)
        x = self.dense_1(x)
        x = tf.nn.relu(x)
        return self.dense_2(x)
    
is_ef = True
backbone_name = 'efficientnet-b0'

N_TILES = 48
IMG_SIZE = 256

if backbone_name.startswith('efficientnet'):
    model_fn = getattr(efn, f'EfficientNetB{backbone_name[-1]}')

    
model = ConvNet(engine=model_fn, input_shape=(IMG_SIZE, IMG_SIZE, 3), weights=None)



TRAIN = '../input/prostate-cancer-grade-assessment/train_images/'
MASKS = '../input/prostate-cancer-grade-assessment/train_label_masks/'
BASE_PATH = '../input/prostate-cancer-grade-assessment/'
train = pd.read_csv(BASE_PATH + "train.csv")
train.head()

sub = pd.read_csv("../input/prostate-cancer-grade-assessment/sample_submission.csv")
sub.head()

test = pd.read_csv("../input/prostate-cancer-grade-assessment/test.csv")
test.head()

TEST = '../input/prostate-cancer-grade-assessment/test_images/'


PRED_PATH = TEST 
df = sub
t_df = test




####


transforms_val0 = albumentations.Compose([])


transforms_val1 = albumentations.Compose([
    albumentations.VerticalFlip(p=1)
])

transforms_val2= albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.VerticalFlip(p=0.5),    
])

transforms_val3= albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.VerticalFlip(p=0.5),    
])



n_TTA = 2        
dummy_data = tf.zeros((n_TTA * N_TILES, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32)
_ = model(dummy_data)        
        

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(lr= 1e-05), loss= tf.nn.sigmoid_cross_entropy_with_logits)
model.load_weights('../input/pandaenetb042x256x256x3/efficientnet-b0-48-full-epochs60.h5')

####

if os.path.exists(PRED_PATH):
    predictions10 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image) 
        isup = 0.5*np.sum(pred)
        predictions10.append(isup)

        del patches, img
        gc.collect()

else:
    PRED_PATH = TRAIN
    df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    df = df_train.loc[:10]
    df = df[['image_id','isup_grade']].copy()
    predictions10 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image) 
        isup = 0.5*np.sum(pred)
        predictions10.append(isup)

        del patches, img
        gc.collect()


####

if os.path.exists(PRED_PATH):
    predictions12 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile2(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image) 
        isup = 0.5*np.sum(pred)
        predictions12.append(isup)

        del patches, img
        gc.collect()

else:
    PRED_PATH = TRAIN
    df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    df = df_train.loc[:10]
    df = df[['image_id','isup_grade']].copy()
    predictions12 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile2(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image) 
        isup = 0.5*np.sum(pred)
        predictions12.append(isup)

        del patches, img
        gc.collect()

In [ ]:

class ConvNet(tf.keras.Model):

    def __init__(self, engine, input_shape, weights):
        super(ConvNet, self).__init__()
        
        self.engine = engine(
            include_top=False, input_shape=input_shape, weights=weights)
        
        
        self.avg_pool2d = tf.keras.layers.GlobalAveragePooling2D()
        self.dropout = tf.keras.layers.Dropout(0.5)
        self.dense_1 = tf.keras.layers.Dense(1024)
        self.dense_2 = tf.keras.layers.Dense(5,activation='sigmoid')

    @tf.function
    def call(self, inputs, **kwargs):
        x = tf.reshape(inputs, (-1, IMG_SIZE, IMG_SIZE, 3))
        x = self.engine(x)
        shape = x.shape
        x = tf.reshape(x, (-1, N_TILES, shape[1], shape[2], shape[3])) 
        x = tf.transpose(x, perm=[0, 2, 1, 3, 4])
        x = tf.reshape(x, (-1, shape[1], N_TILES*shape[2], shape[3])) 
        x = self.avg_pool2d(x)
        x = self.dropout(x, training=False)
        x = self.dense_1(x)
        x = tf.nn.relu(x)
        return self.dense_2(x)
    
is_ef = True
backbone_name = 'efficientnet-b1'
N_TILES = 48
IMG_SIZE = 256


if backbone_name.startswith('efficientnet'):
    model_fn = getattr(efn, f'EfficientNetB{backbone_name[-1]}')
    
model = ConvNet(engine=model_fn, input_shape=(IMG_SIZE, IMG_SIZE, 3), weights=None)




n_TTA = 2        
dummy_data = tf.zeros((n_TTA * N_TILES, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32)
_ = model(dummy_data)        


TRAIN = '../input/prostate-cancer-grade-assessment/train_images/'
MASKS = '../input/prostate-cancer-grade-assessment/train_label_masks/'
BASE_PATH = '../input/prostate-cancer-grade-assessment/'
train = pd.read_csv(BASE_PATH + "train.csv")
train.head()

sub = pd.read_csv("../input/prostate-cancer-grade-assessment/sample_submission.csv")
sub.head()

test = pd.read_csv("../input/prostate-cancer-grade-assessment/test.csv")
test.head()

TEST = '../input/prostate-cancer-grade-assessment/test_images/'


PRED_PATH = TEST 
df = sub
t_df = test

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(lr= 1e-05), loss= tf.nn.sigmoid_cross_entropy_with_logits)
model.load_weights('../input/pandaenetb042x256x256x3/efficientnet-b1-48-full-epochs60.h5')

####

if os.path.exists(PRED_PATH):
    predictions20 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image) 
        isup = 0.5*np.sum(pred)
        predictions20.append(isup)

        del patches, img
        gc.collect()

else:
    PRED_PATH = TRAIN
    df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    df = df_train.loc[:10]
    df = df[['image_id','isup_grade']].copy()
    predictions20 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image) 
        isup = 0.5*np.sum(pred)
        predictions20.append(isup)

        del patches, img
        gc.collect()


####

if os.path.exists(PRED_PATH):
    predictions22 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile2(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image) 
        isup = 0.5*np.sum(pred)
        predictions22.append(isup)

        del patches, img
        gc.collect()

else:
    PRED_PATH = TRAIN
    df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    df = df_train.loc[:10]
    df = df[['image_id','isup_grade']].copy()
    predictions22 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile2(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image) 
        isup = 0.5*np.sum(pred)
        predictions22.append(isup)

        del patches, img
        gc.collect()

In [ ]:

class ConvNet(tf.keras.Model):

    def __init__(self, engine, input_shape, weights):
        super(ConvNet, self).__init__()
        
        self.engine = engine(
            include_top=False, input_shape=input_shape, weights=weights)
        
        
        self.avg_pool2d = tf.keras.layers.GlobalAveragePooling2D()
        self.dropout = tf.keras.layers.Dropout(0.5)
        self.dense_1 = tf.keras.layers.Dense(1024)
        self.dense_2 = tf.keras.layers.Dense(5,activation='sigmoid')

    @tf.function
    def call(self, inputs, **kwargs):
        x = tf.reshape(inputs, (-1, IMG_SIZE, IMG_SIZE, 3))
        x = self.engine(x)
        shape = x.shape
        x = tf.reshape(x, (-1, N_TILES, shape[1], shape[2], shape[3])) 
        x = tf.transpose(x, perm=[0, 2, 1, 3, 4])
        x = tf.reshape(x, (-1, shape[1], N_TILES*shape[2], shape[3])) 
        x = self.avg_pool2d(x)
        x = self.dropout(x, training=False)
        x = self.dense_1(x)
        x = tf.nn.relu(x)
        return self.dense_2(x)
    
is_ef = True
backbone_name = 'efficientnet-b2'
N_TILES = 48
IMG_SIZE = 256


if backbone_name.startswith('efficientnet'):
    model_fn = getattr(efn, f'EfficientNetB{backbone_name[-1]}')
    
model = ConvNet(engine=model_fn, input_shape=(IMG_SIZE, IMG_SIZE, 3), weights=None)


n_TTA = 2        
dummy_data = tf.zeros((n_TTA * N_TILES, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32)
_ = model(dummy_data)           



TRAIN = '../input/prostate-cancer-grade-assessment/train_images/'
MASKS = '../input/prostate-cancer-grade-assessment/train_label_masks/'
BASE_PATH = '../input/prostate-cancer-grade-assessment/'
train = pd.read_csv(BASE_PATH + "train.csv")
train.head()

sub = pd.read_csv("../input/prostate-cancer-grade-assessment/sample_submission.csv")
sub.head()

test = pd.read_csv("../input/prostate-cancer-grade-assessment/test.csv")
test.head()

TEST = '../input/prostate-cancer-grade-assessment/test_images/'


PRED_PATH = TEST 
df = sub
t_df = test

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(lr= 1e-05), loss= tf.nn.sigmoid_cross_entropy_with_logits)
model.load_weights('../input/pandaenetb042x256x256x3/efficientnet-b2-48-full-epochs60.h5')

####

if os.path.exists(PRED_PATH):
    predictions30 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image) 
        isup = 0.5*np.sum(pred)
        predictions30.append(isup)

        del patches, img
        gc.collect()

else:
    PRED_PATH = TRAIN
    df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    df = df_train.loc[:10]
    df = df[['image_id','isup_grade']].copy()
    predictions30 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image) 
        isup = 0.5*np.sum(pred)
        predictions30.append(isup)

        del patches, img
        gc.collect()


####

if os.path.exists(PRED_PATH):
    predictions32 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile2(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image) 
        isup = 0.5*np.sum(pred)
        predictions32.append(isup)

        del patches, img
        gc.collect()

else:
    PRED_PATH = TRAIN
    df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    df = df_train.loc[:10]
    df = df[['image_id','isup_grade']].copy()
    predictions32 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile2(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image) 
        isup = 0.5*np.sum(pred)
        predictions32.append(isup)

        del patches, img
        gc.collect()
        


In [ ]:

from tensorflow.keras.applications import densenet as den



class ConvNet(tf.keras.Model):

    def __init__(self, engine, input_shape, weights):
        super(ConvNet, self).__init__()
        
        self.engine = engine(
            include_top=False, input_shape=input_shape, weights=weights)
        
        
        self.avg_pool2d = tf.keras.layers.GlobalAveragePooling2D()
        self.dropout = tf.keras.layers.Dropout(0.5)
        self.dense_1 = tf.keras.layers.Dense(1024)
        self.dense_2 = tf.keras.layers.Dense(5,activation='sigmoid')

    @tf.function
    def call(self, inputs, **kwargs):
        x = tf.reshape(inputs, (-1, IMG_SIZE, IMG_SIZE, 3))
        x = self.engine(x)
        shape = x.shape
        x = tf.reshape(x, (-1, N_TILES, shape[1], shape[2], shape[3])) 
        x = tf.transpose(x, perm=[0, 2, 1, 3, 4])
        x = tf.reshape(x, (-1, shape[1], N_TILES*shape[2], shape[3])) 
        x = self.avg_pool2d(x)
        x = self.dropout(x, training=False)
        x = self.dense_1(x)
        x = tf.nn.relu(x)
        return self.dense_2(x)
    


N_TILES = 48
IMG_SIZE = 256



model_fn = getattr(den, 'DenseNet121')
    
model = ConvNet(engine=model_fn, input_shape=(IMG_SIZE, IMG_SIZE, 3), weights=None)


n_TTA = 4        
dummy_data = tf.zeros((n_TTA * N_TILES, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32)
_ = model(dummy_data)      



TRAIN = '../input/prostate-cancer-grade-assessment/train_images/'
MASKS = '../input/prostate-cancer-grade-assessment/train_label_masks/'
BASE_PATH = '../input/prostate-cancer-grade-assessment/'
train = pd.read_csv(BASE_PATH + "train.csv")
train.head()

sub = pd.read_csv("../input/prostate-cancer-grade-assessment/sample_submission.csv")
sub.head()

test = pd.read_csv("../input/prostate-cancer-grade-assessment/test.csv")
test.head()

TEST = '../input/prostate-cancer-grade-assessment/test_images/'


PRED_PATH = TEST 
df = sub
t_df = test

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(lr= 1e-05), loss= tf.nn.sigmoid_cross_entropy_with_logits)
model.load_weights('../input/pandaenetb042x256x256x3/DenseNet121-48-full-epochs60.h5')

####

if os.path.exists(PRED_PATH):
    predictions40 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile(img)
        patches1 = patches.copy()
        patches2 = patches.copy()
        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches1[k, ] = transforms_val0(image=patches1[k, ])['image']
            patches2[k, ] = transforms_val1(image=patches2[k, ])['image']
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches1, patches2, patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image) 
        isup = 0.25*np.sum(pred)
        predictions40.append(isup)

        del patches, img
        gc.collect()

else:
    PRED_PATH = TRAIN
    df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    df = df_train.loc[:10]
    df = df[['image_id','isup_grade']].copy()
    predictions40 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile(img)
        patches1 = patches.copy()
        patches2 = patches.copy()
        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches1[k, ] = transforms_val0(image=patches1[k, ])['image']
            patches2[k, ] = transforms_val1(image=patches2[k, ])['image']
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches1, patches2, patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image) 
        isup = 0.25*np.sum(pred)
        predictions40.append(isup)

        del patches, img
        gc.collect()


####

if os.path.exists(PRED_PATH):
    predictions42 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile2(img)
        patches1 = patches.copy()
        patches2 = patches.copy()
        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches1[k, ] = transforms_val0(image=patches1[k, ])['image']
            patches2[k, ] = transforms_val1(image=patches2[k, ])['image']
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches1, patches2, patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image) 
        isup = 0.25*np.sum(pred)
        predictions42.append(isup)

        del patches, img
        gc.collect()

else:
    PRED_PATH = TRAIN
    df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    df = df_train.loc[:10]
    df = df[['image_id','isup_grade']].copy()
    predictions42 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile2(img)
        patches1 = patches.copy()
        patches2 = patches.copy()
        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches1[k, ] = transforms_val0(image=patches1[k, ])['image']
            patches2[k, ] = transforms_val1(image=patches2[k, ])['image']
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches1, patches2, patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image) 
        isup = 0.25*np.sum(pred)
        predictions42.append(isup)

        del patches, img
        gc.collect()


        
del model, dummy_data, sub, pred, train, isup, image
del patches1,patches2,patches3,patches4    

gc.collect()  

In [ ]:

sz = 256
N = 42
def tile(img):
    result = []
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    img = img[idxs]
    return img

def tile2(img):
    result = []
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz + ((sz * 2) // 2), (sz - shape[1]%sz)%sz + ((sz * 2) // 2)
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    img = img[idxs]
    return img




class ConvNet(tf.keras.Model):

    def __init__(self, engine, input_shape, weights):
        super(ConvNet, self).__init__()
        
        self.engine = engine(
            include_top=False, input_shape=input_shape, weights=weights)
        
        
        self.avg_pool2d = tf.keras.layers.GlobalAveragePooling2D()
        self.dropout = tf.keras.layers.Dropout(0.5)
        self.dense_1 = tf.keras.layers.Dense(1024)
        self.dense_2 = tf.keras.layers.Dense(1)

    @tf.function
    def call(self, inputs, **kwargs):
        x = tf.reshape(inputs, (-1, IMG_SIZE, IMG_SIZE, 3))
        x = self.engine(x)
        shape = x.shape
        x = tf.reshape(x, (-1, N_TILES, shape[1], shape[2], shape[3])) 
        x = tf.transpose(x, perm=[0, 2, 1, 3, 4])
        x = tf.reshape(x, (-1, shape[1], N_TILES*shape[2], shape[3])) 
        x = self.avg_pool2d(x)
        x = self.dropout(x, training=False)
        x = self.dense_1(x)
        x = tf.nn.relu(x)
        return self.dense_2(x)
    
is_ef = True
backbone_name = 'efficientnet-b0'

N_TILES = 42
IMG_SIZE = 256

if backbone_name.startswith('efficientnet'):
    model_fn = getattr(efn, f'EfficientNetB{backbone_name[-1]}')

    
model = ConvNet(engine=model_fn, input_shape=(IMG_SIZE, IMG_SIZE, 3), weights=None)



TRAIN = '../input/prostate-cancer-grade-assessment/train_images/'
MASKS = '../input/prostate-cancer-grade-assessment/train_label_masks/'
BASE_PATH = '../input/prostate-cancer-grade-assessment/'
train = pd.read_csv(BASE_PATH + "train.csv")
train.head()

sub = pd.read_csv("../input/prostate-cancer-grade-assessment/sample_submission.csv")
sub.head()

test = pd.read_csv("../input/prostate-cancer-grade-assessment/test.csv")
test.head()

TEST = '../input/prostate-cancer-grade-assessment/test_images/'


PRED_PATH = TEST 
df = sub
t_df = test




####





n_TTA = 2        
dummy_data = tf.zeros((n_TTA * N_TILES, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32)
_ = model(dummy_data)   



In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(lr= 1e-05), loss= tf.nn.sigmoid_cross_entropy_with_logits)
model.load_weights('../input/pandaenetb042x256x256x3/efficientnet-b0-fold0-epochs40.h5')

####

if os.path.exists(PRED_PATH):
    predictions50 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image)
        isup = np.mean(pred)
        predictions50.append(isup)

        del patches, img
        gc.collect()

else:
    PRED_PATH = TRAIN
    df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    df = df_train.loc[:10]
    df = df[['image_id','isup_grade']].copy()
    predictions50 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image)
        isup = np.mean(pred)
        predictions50.append(isup)

        del patches, img
        gc.collect()


####

if os.path.exists(PRED_PATH):
    predictions52 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile2(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image)
        isup = np.mean(pred)
        predictions52.append(isup)

        del patches, img
        gc.collect()

else:
    PRED_PATH = TRAIN
    df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    df = df_train.loc[:10]
    df = df[['image_id','isup_grade']].copy()
    predictions52 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile2(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image)
        isup = np.mean(pred)
        predictions52.append(isup)

        del patches, img
        gc.collect()
        


In [ ]:


model.compile(optimizer = tf.keras.optimizers.Adam(lr= 1e-05), loss= tf.nn.sigmoid_cross_entropy_with_logits)
model.load_weights('../input/pandaenetb042x256x256x3/efficientnet-b0-fold4-epochs60.h5')

####

if os.path.exists(PRED_PATH):
    predictions60 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image)
        isup = np.mean(pred)
        predictions60.append(isup)

        del patches, img
        gc.collect()

else:
    PRED_PATH = TRAIN
    df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    df = df_train.loc[:10]
    df = df[['image_id','isup_grade']].copy()
    predictions60 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image)
        isup = np.mean(pred)
        predictions60.append(isup)

        del patches, img
        gc.collect()


####

if os.path.exists(PRED_PATH):
    predictions62 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile2(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image)
        isup = np.mean(pred)
        predictions62.append(isup)

        del patches, img
        gc.collect()

else:
    PRED_PATH = TRAIN
    df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    df = df_train.loc[:10]
    df = df[['image_id','isup_grade']].copy()
    predictions62 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile2(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image)
        isup = np.mean(pred)
        predictions62.append(isup)

        del patches, img
        gc.collect()
        







In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(lr= 1e-05), loss= tf.nn.sigmoid_cross_entropy_with_logits)
model.load_weights('../input/pandaenetb042x256x256x3/efficientnet-b0-fold2-epochs40.h5')

####

if os.path.exists(PRED_PATH):
    predictions70 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image)
        isup = np.mean(pred)
        predictions70.append(isup)

        del patches, img
        gc.collect()

else:
    PRED_PATH = TRAIN
    df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    df = df_train.loc[:10]
    df = df[['image_id','isup_grade']].copy()
    predictions70 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image)
        isup = np.mean(pred)
        predictions70.append(isup)

        del patches, img
        gc.collect()


####

if os.path.exists(PRED_PATH):
    predictions72 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile2(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image)
        isup = np.mean(pred)
        predictions72.append(isup)

        del patches, img
        gc.collect()

else:
    PRED_PATH = TRAIN
    df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    df = df_train.loc[:10]
    df = df[['image_id','isup_grade']].copy()
    predictions72 = []
    for index, row in tqdm(df.iterrows(), total = df.shape[0]):
        
        
        image_id = row['image_id']
        
        img_path = PRED_PATH + image_id + '.tiff' #BASE_PATH
        
        img = skimage.io.MultiImage(img_path)[1]
        
        patches = tile2(img)

        patches3 = patches.copy()
        patches4 = patches.copy() 
        
        k = 0
        while k < N_TILES:
            patches3[k, ] = transforms_val2(image=patches3[k, ])['image']
            patches4[k, ] = transforms_val3(image=patches4[k, ])['image']
            k += 1
        
        image = np.stack([patches3, patches4])
        image = image / 255.0
        
        pred = model.predict(image)
        isup = np.mean(pred)
        predictions72.append(isup)

        del patches, img
        gc.collect()
        



del model, dummy_data, sub, pred, train, isup, image
del patches3,patches4    

gc.collect()  

In [ ]:

PREDS = (1/5)*PREDS + (1/5)*PREDS1 + (1/5)*PREDS2 + (1/5)*PREDS3 + (1/5)*PREDS4

FINAL = np.round( (6/10)*PREDS +
                  (2/60)*np.array(predictions10) + (2/60)*np.array(predictions12) + 
                  (2/60)*np.array(predictions20) + (2/60)*np.array(predictions22) +
                  (2/60)*np.array(predictions30) + (2/60)*np.array(predictions32) +
                  (0.5/10)*np.array(predictions40) + (0.5/10)*np.array(predictions42) +
                  (1/60)*np.array(predictions50) + (1/60)*np.array(predictions52) +
                  (1/60)*np.array(predictions60) + (1/60)*np.array(predictions62) +
                  (1/60)*np.array(predictions70) + (1/60)*np.array(predictions72) )


df['isup_grade'] = FINAL.astype(int)
df[['image_id', 'isup_grade']].to_csv('submission.csv', index=False)
print(df.head())
print()
print(df.isup_grade.value_counts())